In [1]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    ResNet34, EfficientNetV2, build_efficientnetv2,
    Var_ECAPA, MHA_LAS
)
from src.coatnet import CoAtNet

seed_everything(42, workers=True)

Global seed set to 42


42

In [2]:
num_classes = 5
dm = VoxCelebDataModule(
    batch_size=16,
    from_memory=True,
    spec_augment=True
    # data_dir="D:\Datasets\subset"
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="AdamW",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm,
    ckpt_path=checkpoint_path
)

In [3]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    margin=0.2,
    scale=30
)
model = build_efficientnetv2(
    embeddings_dim=512,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="NAdam",
    lr_scheduler="ReducePlateau",
    lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger("tb_logs", name="efficientnetv2")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Loading data for val: 100%|██████████| 27/27 [00:00<00:00, 2454.51it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type        | Params
-------------------------------------------
0 | loss_func  | SoftmaxLoss | 2.6 K 
1 | acc        | Accuracy    | 0     
2 | f1         | F1Score     | 0     
3 | stem       | Sequential  | 264   
4 | middle     | Sequential  | 42.3 M
5 | head       | Sequential  | 657 K 
6 | embeddings | Linear      | 655 K 
7 | clf        | Linear      | 2.6 K 
-------------------------------------------
43.6 M    Trainable params
0         Non-trainable params
43.6 M    Total params
174.472   Total estimated model params size (MB)
D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:631: UserWarning: Checkpoint directory F:\Copia HD G\Universita\UNIMI\Audio Pattern Recognition\Project\checkpoi

Global seed set to 42


Epoch 4:   7%|▋         | 9/131 [00:01<00:21,  5.56it/s, loss=1.59, v_num=4, train_loss_step=1.610, train_acc_step=0.312, train_f1_step=0.243, val_loss_step=1.580, val_acc_step=0.455, val_f1_step=0.284, val_loss_epoch=1.710, val_acc_epoch=0.185, val_f1_epoch=0.116, train_loss_epoch=1.540, train_acc_epoch=0.271, train_f1_epoch=0.235]    

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    margin=0.2,
    scale=30
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = MHA_LAS(
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="AdamW",
    lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger("tb_logs", name="mha_las")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = Var_ECAPA(num_classes=5, loss_func=aams)
logger = TensorBoardLogger("tb_logs", name="var_ecapa")
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)

In [ ]:
from torch import nn

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/coatnet/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)

num_classes = 5
dm = VoxCelebDataModule(batch_size=4)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = CoAtNet(in_ch=1, embeddings_dim=5, image_size=224, num_classes=5, loss_func=ce_loss)
logger = TensorBoardLogger("tb_logs", name="coatnet")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback]
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)